# Desafio de NLP - NeuralMed
- **CANDIDATO:** Pedro Luís Azevedo Costa

# Importações

In [1]:
import pandas as pd
import magic
import unicodedata
from bs4 import BeautifulSoup
from striprtf.striprtf import rtf_to_text
import re

from IPython.display import HTML, display

import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
import random

from spacy.training.example import Example
from sklearn.metrics import precision_score, recall_score, f1_score




# Etapa 1 
- Faça uma análise descritiva em cima dos documentos. O que você conclui apenas olhando para eles? Qual a forma de escrita dos documentos? Há tipos diferentes? Olhando os textos consegue identificar alguma alteração comum entre os pacientes? 

## Carregando o dataset

In [2]:
file_path = "sample_teste_nlp.csv"
df_doc = pd.read_csv(file_path)
df_doc.head()

,patient_id,age,sex,ehr_date,text
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:..."
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:..."
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...


## Análise Exploratória  

O dataset contém 5 colunas:  
- `patient_id`: numérica  
- `age`: numérica  
- `sex`: categórica  
- `ehr_date`: datetime  
- `text`: textual  

Conforme mencionado no enunciado, o foco deste exercício está na coluna `text`, que contém os documentos a serem analisados. Ao observar as primeiras 5 linhas do dataset, é possível identificar três tipos diferentes de formatação de texto: **plain text**, **HTML** e **RTF**.  

Assim, o primeiro passo desta análise preliminar é descobrir todos os tipos possíveis de formatação.  


In [3]:
def detect_mime_type(text):
    """
    Detecta a formatação do texto
    """
    mime = magic.Magic(mime=True)
    return mime.from_buffer(text.encode('utf-8'))

# Aplica a função detect_mime_type na coluna 'text' do dataframe
df_doc['text_type'] = df_doc['text'].apply(lambda x: detect_mime_type(x))

tipos_texto = df_doc['text_type'].unique()

print("Tipos de texto encontrados: ", tipos_texto)

# Quantidade de textos por tipo
count_text_type = df_doc['text_type'].value_counts()

print("Quantidade de textos por tipo: ", count_text_type)


Tipos de texto encontrados:  ['text/html' 'text/rtf' 'text/plain']
Quantidade de textos por tipo:  text_type
text/html     4253
text/rtf      1852
text/plain     729
Name: count, dtype: int64


Conforme mostrado na célula acima, identificamos três tipos de texto, com predominância de **HTML**, que será o foco da análise.  

Embora o ideal fosse realizar uma avaliação cuidadosa de cada tipo de texto, devido às restrições de tempo e recursos, concentraremos nossos esforços no formato **HTML**. Esse formato foi escolhido por sua maior predominância e pela estrutura mais simples de navegar em comparação aos demais.  


## Visualizando Exemplos

In [4]:
# Avaliando alguns textos em HTML

df_filtered_html = df_doc.loc[df_doc['text_type'] == 'text/html']

display(HTML(df_filtered_html['text'].iloc[0]))
display(HTML(df_filtered_html['text'].iloc[100]))
display(HTML(df_filtered_html['text'].iloc[200]))
display(HTML(df_filtered_html['text'].iloc[300]))
display(HTML(df_filtered_html['text'].iloc[400]))
display(HTML(df_filtered_html['text'].iloc[500]))




## Extraindo Padrões

Ao observar os documentos acima, é possível notar que alguns possuem um formato que facilita a anotação de diagnósticos. Há um campo chamado **"Hipótese Diagnóstica"** e, logo após, uma sigla ou o nome completo do diagnóstico.  

Para este trabalho, vamos nos focar nesse atributo.  

Deste modo, o próximo passo é definir uma função capaz de extrair estes rótulos.

In [5]:
df_filtered_html['text'].iloc[400]

'<html tasy="html5"><body><p style="text-align:left;"><span style="font-style: normal; font-variant: normal; font-stretch: normal; font-size: 11pt; font-family: Arial; color: rgb(0, 0, 0); background-color: rgb(255, 255, 255);">\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0<strong>\xa0 ANAMNESE - CENTRO CLÍNICO</strong></span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style="">Queixa Principal: TUMORAÇÃO 2º DEDO PÉ DIR HA TEMPOS</span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;">Alergias: Nega Alergia à Medicamentos</p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style="">HDA:\xa0</span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style="">HPP:\xa0</span></p><p style="text-align:left;">\xa0</p><p style="text-align:left;"><span style=""><span style="">Sinais Vitais</span></span></p><p

In [6]:
def extrair_hipotese_diagnostica(html):
    # Parseia o HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Encontra o texto que vem após 'Hipótese Diagnóstica:'
    texto = soup.get_text()

    # Usando regex para encontrar o texto após 'Hipótese Diagnóstica:'
    padrao = re.compile(r'Hipótese Diagnóstica:([^\n]+)')
    resultado = padrao.search(texto)

    if resultado:
        match_result = resultado.group(1).strip()
        match_result = match_result.replace('\xa0', '||')
        match_result = match_result.replace('\u200b', '||')
        match_result = match_result.split('||')[0]

        return match_result  # Retorna o texto após 'Hipótese Diagnóstica:'
    else:
        return None  # Caso não encontre o padrão

extrair_hipotese_diagnostica(df_filtered_html['text'].iloc[5])

In [7]:
df_doc['hipotese_diagnostica'] = df_doc['text'].apply(lambda x: extrair_hipotese_diagnostica(x))
# Lower case para padronizar o texto
df_doc['hipotese_diagnostica'] = df_doc['hipotese_diagnostica'].str.lower()
df_doc.head()

/tmp/ipykernel_866699/2961999388.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


,patient_id,age,sex,ehr_date,text,text_type,hipotese_diagnostica
0,341196,82.0,M,2023-09-05 10:23:37,"<html tasy=""html5""><body><p style=""text-align:...",text/html,hernia inguinal
1,4309949,32.0,F,2023-11-01 12:10:59,"<html tasy=""html5""><body><p style=""text-align:...",text/html,None
2,404932,91.0,M,2023-09-16 07:24:02,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,text/rtf,None
3,5603035,53.0,F,2024-08-03 06:56:48,RET 6 MESES PARA AVALIAÇÃO CALCIO + VIT D,text/plain,None
4,154876,27.0,M,2023-11-05 04:58:01,{\rtf1\ansi\ansicpg1252{\fonttbl{\f0\fnil\fcha...,text/rtf,None


In [8]:
# Calcula o número total de hipóteses diagnósticas extraídas
total_hipoteses = df_doc['hipotese_diagnostica'].count()

# Calcula o número de hipóteses distintas extraídas|
hipoteses_distintas = df_doc['hipotese_diagnostica'].unique().shape[0]

print("Total de hipóteses diagnósticas extraídas: ", total_hipoteses)

print("Total de hipóteses diagnósticas distintas extraídas: ", hipoteses_distintas)

Total de hipóteses diagnósticas extraídas:  1103
Total de hipóteses diagnósticas distintas extraídas:  753


In [9]:
df_doc['hipotese_diagnostica'].dropna().value_counts().head(20)

hipotese_diagnostica
                                                 45
conduta:                                         40
rotina                                           18
ivas                                             16
has                                              14
r10                                              12
geca                                             12
z10                                              11
r51                                               9
amigdalite                                        9
itu                                               8
gastroenterite                                    6
dispepsia                                         6
sinusite                                          6
dor abdominal                                     5
lombalgia                                         5
z370 - nascimento unico, nativivo [nado-vivo]     5
enxaqueca                                         5
dorsalgia                                  

Ao analisar as 20 hipóteses diagnósticas mais frequentes, é possível identificar alguns erros de extração, como a presença de um campo vazio na primeira posição e a ocorrência do termo "conduta:". Além disso, observa-se que "rotina" aparece como uma hipótese diagnóstica, o que não é apropriado. Portanto, a melhor abordagem para lidar com esses casos é remover as linhas que contêm essas classificações.

In [10]:
df_doc['hipotese_diagnostica'].str.contains('conduta').isna()

0       False
1        True
2        True
3        True
4        True
        ...  
6829     True
6830     True
6831     True
6832     True
6833     True
Name: hipotese_diagnostica, Length: 6834, dtype: bool

In [11]:
# Remove linhas com hipose_diagnostica nula, contém "conduta" e "rotina"
df_doc = df_doc[~(df_doc['hipotese_diagnostica'] == '')]
df_doc = df_doc[~df_doc['hipotese_diagnostica'].str.contains('conduta').isna()]
df_doc = df_doc[~df_doc['hipotese_diagnostica'].str.contains('rotina').isna()]

# Calcula o número total de hipóteses diagnósticas extraídas
total_hipoteses = df_doc['hipotese_diagnostica'].count()

# Calcula o número de hipóteses distintas extraídas|
hipoteses_distintas = df_doc['hipotese_diagnostica'].unique().shape[0]

print("Total de hipóteses diagnósticas extraídas: ", total_hipoteses)

print("Total de hipóteses diagnósticas distintas extraídas: ", hipoteses_distintas)

display(df_doc['hipotese_diagnostica'].dropna().value_counts().head(20))

Total de hipóteses diagnósticas extraídas:  1058
Total de hipóteses diagnósticas distintas extraídas:  751


hipotese_diagnostica
conduta:                                         40
rotina                                           18
ivas                                             16
has                                              14
r10                                              12
geca                                             12
z10                                              11
r51                                               9
amigdalite                                        9
itu                                               8
gastroenterite                                    6
dispepsia                                         6
sinusite                                          6
dor abdominal                                     5
dorsalgia                                         5
z370 - nascimento unico, nativivo [nado-vivo]     5
enxaqueca                                         5
lombalgia                                         5
amigdalite aguda                           

Com base na tabela acima, conclui-se que os diagnósticos mais comuns nos documentos analisados são **IVAS** e **HAS**, siglas que correspondem a "Infecções Agudas das Vias Aéreas Superiores" e "Hipertensão Arterial Sistêmica".


# Etapa 2
- Como você extrairia informações de medicamentos, exames e doenças do paciente? Analise os dados que tem em mãos e descreva o pipeline de desenvolvimento para um modelo como esses, levando em consideração todo o processo de análise, anotação, treino até a validação do modelo. 

A etapa anterior abordou a extração de hipóteses diagnósticas. Nesta etapa, o foco será preparar o conjunto de dados no formato necessário para treinar um modelo no spaCy. Além disso, o conjunto de dados será dividido em três partes: treino, validação e teste, garantindo uma estrutura adequada para a avaliação do modelo.

O primeiro passo para prepara o conjunto de dados é extrair o texto do formato html de modo a convertê-lo em plain text

In [12]:
# Função que limpa todo o html e deixa apenas o plain text
def clean_html(text):
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(separator = " ")
    #text = re.sub(r'\u200b', ' ', text)
    return text

def clean_special_characters(texto):
    # Normaliza o texto para remover caracteres compostos
    texto_normalizado = unicodedata.normalize('NFKD', texto)
    # Remove caracteres de controle e espaços invisíveis
    texto_limpo = ''.join(c for c in texto_normalizado if c.isprintable())
    return texto_limpo.strip()

def map_clean_text_type(row):
    if row['text_type'] == 'text/html':
        clean_text = clean_html(row['text'])
        clean_text = clean_special_characters(clean_text)
        return clean_text.lower()
    else:
        return row['text'].lower()
    
df_doc_filtered = df_doc.loc[df_doc['text_type'] == 'text/html']
df_doc_filtered['processed_text'] = df_doc_filtered.apply(map_clean_text_type, axis=1)
# Converte o texto de html para plain text
df_doc_filtered[['processed_text', 'text']].head()


,processed_text,text
0,anamnese - centro clínico queixa principal:...,"<html tasy=""html5""><body><p style=""text-align:..."
6,anamnese - centro clínico queixa principal:...,"<html tasy=""html5""><body><p style=""text-align:..."
7,anamnese/exame físico - ps anamnese: queixa...,"<html tasy=""html5""><body><p style=""text-align:..."
10,evolução de enfermagem 6 o dih hipótes...,"<html tasy=""html5""><body><p style=""text-align:..."
15,anamnese/exame físico - ps queixa principal:...,"<html tasy=""html5""><body><p><span style=""""><sp..."


In [13]:
# Função para transformar o DataFrame em um formato SpaCy treinável
def create_training_data(df):
    training_data = []
    for text, entity in zip(df["processed_text"], df["hipotese_diagnostica"]):
        start = text.find(entity)
        if start != -1:  # Certifica de que a entidade existe no texto
            end = start + len(entity)
            training_data.append((text, {"entities": [(start, end, "HIPOTESE_DIAGNOSTICA")]}))
    return training_data


In [14]:
# Criar os dados de treinamento no formato SpaCy
training_data = create_training_data(df_doc_filtered)

# Separa em treino, teste e validação
random.shuffle(training_data)
train_data = training_data[:int(len(training_data) * 0.8)]
valid_data = training_data[int(len(training_data) * 0.8):int(len(training_data) * 0.9)]
test_data = training_data[int(len(training_data) * 0.9):]





In [15]:
# Função para salvar os dados em formato binário do SpaCy
def save_training_data(training_data, output_path="training_data.spacy"):
    nlp = spacy.blank("pt")  # Usando modelo base em português
    db = DocBin()  # Objeto para armazenar dados
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annotations["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents  # Adiciona as entidades no Doc
        db.add(doc)
    db.to_disk(output_path)
    print(f"Dados de treinamento salvos em {output_path}")

# Salvar os conjuntos de treino, validação e teste
save_training_data(train_data, "train_data.spacy")
save_training_data(valid_data, "val_data.spacy")
save_training_data(test_data, "test_data.spacy")


Dados de treinamento salvos em train_data.spacy
Dados de treinamento salvos em val_data.spacy
Dados de treinamento salvos em test_data.spacy


In [24]:
!python -m spacy train config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./val_data.spacy 

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-01-18 18:52:50,015] [WARNING] [W112] The model specified to use for initial vectors (pt_core_news_sm) has no vectors. This is almost certainly a mistake.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.0005
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.33    0.00    0.00    0.00    0.00
  0      50          1.01   1343.37    0.00    0.00    0.00    0.00
  0     100          0.52     69.43   49.68   51.32   48.15    0.50
  0     150          0.95     56.29   50.93   51.25   50.62    0.51
  0     200          3.36     93.15   61.54   59.09   64.20    0.62
  0     250          1.46     61.41   56.47   53.93   59.26    0.56
  0   

In [17]:
# Carrega o modelo treinado
nlp = spacy.load("./output/model-best")

/home/ubuntu/Installations/anaconda3/envs/spacy/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'pt_pipeline' (0.0.0) was trained with spaCy v3.7.5 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [18]:
# Faz a predição em um texto
doc = nlp(test_data[4][0])

# Exibe as entidades preditas
for ent in doc.ents:
    print(ent.text, ent.label_)

In [19]:
# PRINTA O EXEMPLO DE HIPOTESE DIAGNOSTICA
print(df_doc_filtered['hipotese_diagnostica'].iloc[1])
print(df_doc_filtered['processed_text'].iloc[0])

dm2
anamnese - centro clínico   queixa principal: dor inguinoescrotal esq a/e   alergias: nega alergia à medicamentos   hda:    hpp:    sinais vitais pa:    x     fc:      fr:       t:        dor:       exame físico:   hipótese diagnóstica:hernia inguinal   conduta:exames


In [20]:
test_data[0][0]

'anamnese de admissão ps adulto   queixa principal :    hda: paciente refere dor em baixo ventre com irradiação para flanco esquerdo há 1 dia. associado a nauseas. nega febre, vomitos, constipação, disuria e outros sintomas.  dum : 25/01/2024  hpp: teratoma em ovario esquerdo  medicações de uso regular : alergias:  nega    sinais vitais: pa:   x   fc:   fr:    sato2:    dor:   temperatura:   exame físico : beg, aaa, lote, eupneica, normocorada, hidratada abd: plano, flacido , dor a palpação em hipogastrio , fossa iliaca direita e flanco direito , sem sinais de irritação peritoneal, db -  ext: sem edemas  hipótese diagnóstica : conduta:   - prescrevo medicação - solicito exames'

In [21]:
# Printa um exemplo do conjunto de teste